In [1]:
import os
import gc
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm
from osgeo import gdal
from rasterstats import zonal_stats

In [ ]:
res = 500
GHS = 'GHS_BUILT_H'
indicator_list = {'AGBH':'Average of the Gross Building Height','ANBH':'Average of the Net Building Height'}
for Character in tqdm(['AGBH','ANBH']):
    read_folder = r'D:\0-data' + '\\' + GHS + '\\' + Character
    raster_list = []
    # generate a list of all raster tiles
    for file in os.listdir(read_folder):
        if file.endswith('.tif'):
            raster_list.append(read_folder + '\\' + file)
    # merge the raster tiles into one
    save_folder = r'D:\0-data\GHS_BUILT_eu' + '\\' + GHS + '\\' + Character
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    save_path = save_folder + '\\' + 'GHS_BUILT_C_2018_'+ Character + '_' + str(res) + '.tif'
    # set resampling method to average, because this is a dataset with continuous values
    gdal.Warp(save_path, raster_list, format='GTiff', dstSRS='EPSG:3035', xRes=res, yRes=res, 
              outputType=gdal.GDT_Float32, resampleAlg='average', options=['COMPRESS=LZW'])

    # Zonal statistics looping all years of NUTS regions
    nuts_folder = r'C:\1-Data\NUTS'
    for nuts_file in tqdm(os.listdir(nuts_folder)):
        if nuts_file.endswith('.shp'):
            nuts_path = nuts_folder + '\\' + nuts_file
            nuts = gpd.read_file(nuts_path)
            nuts_year = nuts_file.split('_')[-2]
            df_comb = None
            for tif in os.listdir(save_folder):
                if tif.endswith('.tif'):
                    # Conduct zonal statistics and add other needed information
                    zs_temp = pd.DataFrame(zonal_stats(nuts_path, save_folder + '\\' + tif, stats="mean"))
                    df_temp = pd.merge(nuts[['NUTS_ID']], zs_temp, left_index=True, right_index=True)
                    df_temp.columns = ['geo','ObsValue']
                    df_temp['Indicator'] = indicator_list[Character]
                    df_temp['freq'] = 'NA'
                    df_temp['ObsTime'] = '2018'
                    df_temp['unit'] = 'm'
                    df_temp['ObsValue'] = df_temp['ObsValue'].apply(lambda x: x*(res/1000)**2)
                    df_comb = df_temp if df_comb is None else pd.concat([df_comb, df_temp])
            geo_source = 'NUTS' + nuts_year
            df_comb['geo_source'] = geo_source
            df_comb.set_index('geo').to_csv(r'D:\1-Case studies\GHS' + '\\' + GHS + '_' + Character + '_' + geo_source +'.csv') 

 65%|████████████████████████████████████████████████████▉                             | 20/31 [04:44<02:29, 13.61s/it]